# Custom CNN model for Brain Tumour Classification

In [1]:
# Import libraries
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score


# hyper-parameters
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
MOMENTUM = 0.9

# Base Path
BASE_PATH = 'archive/'

### Set up GPU growth

In [2]:
device = 'mps' if torch.backends.mps.is_available() else "cpu"

### Define the model architecture

In [3]:
# Create a basic CNN model to try classify the data
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # Max pooling layer
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layer
        self.fc = nn.Linear(32 * 56 * 56, 10)  # Modify the output size based on your task

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x

### Prepare the data for the training of the model

In [4]:
# Create transformer function to create the data-set for the training
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Transform data to Tensor object to load to CNN model
data_transforms = {
    'train': transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
    'test':  transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
}


# Load the training and testing dataset 
image_datasets = {
    'train': torchvision.datasets.ImageFolder(BASE_PATH + 'Training', data_transforms['train']),
    'test':  torchvision.datasets.ImageFolder(BASE_PATH + 'Testing',  data_transforms['test'])
}

# Create the Dataloader object 
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=128,shuffle=True),
    'test' : torch.utils.data.DataLoader(image_datasets['test'],  batch_size=32, shuffle=False)  
}

### Initialize the CNN model 

In [5]:
# Initialize the model
model = MyCNN().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

In [6]:
# Define the training procedure
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    accuracy = correct_predictions / total_predictions
    return running_loss / len(dataloader), accuracy

# Define the testing procedure
from sklearn.metrics import confusion_matrix, classification_report

def evaluate(model, dataloader, criterion, device):
    model.eval()
    predictions = []
    true_labels = []
    test_outputs = []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())
            test_outputs.append(outputs)  # Move to CPU

    test_outputs = torch.cat(test_outputs, dim=0)  # Concatenate the outputs tensor

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    test_loss = criterion(test_outputs, torch.tensor(true_labels).to(device))

    return accuracy, precision, recall, test_loss, true_labels, predictions

### Train the model on the training data

In [7]:
file_training = open('train_cnn.txt', 'a')

start_time = time.time()

for epoch in range(EPOCHS):
    train_loss, train_accuracy = train(model, dataloaders['train'], criterion, optimizer, device)
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.4f}")
    file_training.write('Epoch: {}/{} '.format(epoch+1, EPOCHS) + 'Train Loss: {0} - Train Accuracy: {1} '.format(train_loss, train_accuracy) + '\n' )

finish_time = time.time()

file_training.write('Time needed to train for {0} epochs is: {1}'.format(EPOCHS, finish_time-start_time))
file_training.close()
torch.save(model.state_dict(), 'model-cnn.pth')

Epoch: 1/25
Train Loss: 1.3461 - Train Accuracy: 0.4804
Epoch: 2/25
Train Loss: 0.8146 - Train Accuracy: 0.6884
Epoch: 3/25
Train Loss: 0.6613 - Train Accuracy: 0.7502
Epoch: 4/25
Train Loss: 0.6150 - Train Accuracy: 0.7659
Epoch: 5/25
Train Loss: 0.5536 - Train Accuracy: 0.7981
Epoch: 6/25
Train Loss: 0.5214 - Train Accuracy: 0.8109
Epoch: 7/25
Train Loss: 0.4899 - Train Accuracy: 0.8246
Epoch: 8/25
Train Loss: 0.4508 - Train Accuracy: 0.8447
Epoch: 9/25
Train Loss: 0.4279 - Train Accuracy: 0.8480
Epoch: 10/25
Train Loss: 0.4052 - Train Accuracy: 0.8589
Epoch: 11/25
Train Loss: 0.3782 - Train Accuracy: 0.8683
Epoch: 12/25
Train Loss: 0.3614 - Train Accuracy: 0.8750
Epoch: 13/25
Train Loss: 0.3520 - Train Accuracy: 0.8750
Epoch: 14/25
Train Loss: 0.3243 - Train Accuracy: 0.8871
Epoch: 15/25
Train Loss: 0.3384 - Train Accuracy: 0.8806
Epoch: 16/25
Train Loss: 0.3394 - Train Accuracy: 0.8741
Epoch: 17/25
Train Loss: 0.3102 - Train Accuracy: 0.8883
Epoch: 18/25
Train Loss: 0.2859 - Train 

In [8]:
# Load the model and evaluate the it on the test data
test_file = open('test_cnn.txt', 'a')
loaded_model = MyCNN()
loaded_model.load_state_dict(torch.load('model-cnn.pth'))
loaded_model = loaded_model.to(device)
loaded_model.eval()

# Calculate confusion matrix
test_accuracy, test_precision, test_recall, test_loss, test_true_labels, test_predictions = evaluate(model, dataloaders['test'], criterion, device)

# Calculate Confusion Matrix
confusion_mtx = confusion_matrix(test_true_labels, test_predictions)

# Print Confusion Matrix
print("Confusion Matrix:")
print(confusion_mtx)
test_file.write('Confusion Matrix:\n{}\n'.format(confusion_mtx))

# Calculate Classification Report
class_report = classification_report(test_true_labels, test_predictions)

# Print Classification Report
print("Classification Report:")
print(class_report)
test_file.write('Classification Report:\n{}\n'.format(class_report))
test_file.close()

Confusion Matrix:
[[230  62   2   6]
 [ 16 255  26   9]
 [  1   8 396   0]
 [  4  16   2 278]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.77      0.83       300
           1       0.75      0.83      0.79       306
           2       0.93      0.98      0.95       405
           3       0.95      0.93      0.94       300

    accuracy                           0.88      1311
   macro avg       0.89      0.88      0.88      1311
weighted avg       0.89      0.88      0.88      1311

